# Build The Dataframe

## Import the necessary libraries

In [1]:
import pandas as pd 
import requests

Now use the requests to get the HTML for the Canada postal codes

In [2]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
htmltext = url.text
# print(htmltext)

Using Beautiful Soup it allows use to find the table that contains the 
postal code.

In [3]:
from bs4 import BeautifulSoup
import bs4

soup = bs4.BeautifulSoup(htmltext, 'lxml')
table = soup.find('table','wikitable sortable')
#print (table)

   Now we need to parse the table into columns and store in the dataframe

In [4]:
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text().replace('\n', ''))
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
            print (column_names)
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1

['Postcode', 'Borough', 'Neighbourhood']


We need to remove the newline in order to get the proper
column and data

In [5]:
df = df.replace('\n','', regex=True)

Only keep assigned borough names

In [6]:
df = df[df.Borough != "Not assigned"]

Now we group by postal code and borough and comma delimit all of the neighborhoos